In this experiment, you will train models to distringuish examples of two different genres of Shakespeare's plays: comedies and tragedies. (We'll ignore the histories, sonnets, etc.) Since he died four hundred years ago, Shakespeare has not written any more plays—although scraps of various other works have come to light. We are not, therefore, interested in building models simply to help categorize an unbounded stream of future documents, as we might be in other applications of text classification; rather, we are interested in what a classifier might have to tell us about what we mean by the terms “comedy” and “tragedy”.

You will start by copying and running your `createBasicFeatures` function from the experiment with movie reviews. Do the features the classifier focuses on tell you much about comedy and tragedy in general?

You will then implement another featurization function `createInterestingFeatures`, which will focus on only those features you think are informative for distinguishing between comedy and tragedy. Accuracy on leave-one-out cross-validation may go up, but it more important to look at the features given the highest weight by the classifier. Interpretability in machine learning, of course, may be harder to define than accuracy—although accuracy at some tasks such as summarization is hard enoough.

In [52]:
import json
import requests
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate,LeaveOneOut
import numpy as np
import nltk
from nltk.stem import PorterStemmer,WordNetLemmatizer
from nltk.util import ngrams
from nltk.tag import pos_tag
from nltk.corpus import stopwords
nltk.download('punkt')
import sklearn
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
#read in the shakespeare corpus
def readShakespeare():
  raw = requests.get("https://raw.githubusercontent.com/mutherr/CS6120-PS1-data/master/shakespeare_plays.json").text.strip()
  corpus = [json.loads(line) for line in raw.split("\n")]

  #remove histories from the data, as we're only working with tragedies and comedies
  corpus = [entry for entry in corpus if entry["genre"] != "history"]
  return corpus

This is where you will implement two functions to featurize the data:

In [111]:
#NB: The current contents are for testing only
#This function should return: 
#  -a sparse numpy matrix of document features
#  -a list of the correct genre for each document
#  -a list of the vocabulary used by the features, such that the ith term of the
#    list is the word whose counts appear in the ith column of the matrix. 

# This function should create a feature representation using all tokens that
# contain an alphabetic character.
def createBasicFeatures(corpus):
  #Your code here
  text = [ doc['text'] for doc in corpus ]
  v = CountVectorizer(token_pattern=r'\b[a-zA-Z]{1,}\b',)
  X = v.fit_transform(text)
  vocab = v.get_feature_names()
  texts = X.toarray()
  genres = list()
  for idx in range(len(corpus)):
      genres.append(corpus[idx]["genre"])
  return texts,genres,vocab

# This function can add other features you want that help classification
# accuracy, such as bigrams, word prefixes and suffixes, etc.
def createInterestingFeatures(corpus):
  #Your code here
  text = [ doc['text'] for doc in corpus ]
  token_txt = [ nltk.word_tokenize(i) for i in text]
  word_lemma = WordNetLemmatizer()
  lemmatized_words = []

  for tkn in token_txt:
    lemma = [word_lemma.lemmatize(w).lower() for w in tkn if w not in set(stopwords.words('english'))] #Performed Lemmatization
    pos = pos_tag(lemma) # Applied Parts of Speech Tagging 
    tags = ['NNP','DT','PRP','PRP$','NNPS','IN','NN','FW'] 
    cleaned_text = [word for (word,tag) in pos if tag not in tags] # Identified and removed Nouns,Determinants, Foreign words, Prepositions
    processed_text = " ".join(cleaned_text)
    lemmatized_words.append(processed_text)
    
  v = CountVectorizer(token_pattern=r'[a-zA-Z]{1,}',ngram_range=(1,3))
  texts = v.fit_transform(lemmatized_words)
  vocab = v.get_feature_names()
  genres = list()
  for idx in range(len(corpus)):
      genres.append(corpus[idx]["genre"])
  return texts,genres,vocab

In [11]:
#given a numpy matrix representation of the features for the training set, the 
# vector of true classes for each example, and the vocabulary as described 
# above, this computes the accuracy of the model using leave one out cross 
# validation and reports the most indicative features for each class
def evaluateModel(X,y,vocab,penalty="l1"):
  #create and fit the model
  model = LogisticRegression(penalty=penalty,solver="liblinear")
  results = cross_validate(model,X,y,cv=LeaveOneOut())
  
  #determine the average accuracy
  scores = results["test_score"]
  avg_score = sum(scores)/len(scores)
  
  #determine the most informative features
  # this requires us to fit the model to everything, because we need a
  # single model to draw coefficients from, rather than 26
  model.fit(X,y)
  neg_class_prob_sorted = model.coef_[0, :].argsort()
  pos_class_prob_sorted = (-model.coef_[0, :]).argsort()

  termsToTake = 20
  pos_indicators = [vocab[i] for i in neg_class_prob_sorted[:termsToTake]]
  neg_indicators = [vocab[i] for i in pos_class_prob_sorted[:termsToTake]]

  return avg_score,pos_indicators,neg_indicators

def runEvaluation(X,y,vocab):
  print("----------L1 Norm-----------")
  avg_score,pos_indicators,neg_indicators = evaluateModel(X,y,vocab,"l1")
  print("The model's average accuracy is %f"%avg_score)
  print("The most informative terms for pos are: %s"%pos_indicators)
  print("The most informative terms for neg are: %s"%neg_indicators)
  #this call will fit a model with L2 normalization
  print("----------L2 Norm-----------")
  avg_score,pos_indicators,neg_indicators = evaluateModel(X,y,vocab,"l2")
  print("The model's average accuracy is %f"%avg_score)
  print("The most informative terms for pos are: %s"%pos_indicators)
  print("The most informative terms for neg are: %s"%neg_indicators)
  

In [6]:
corpus = readShakespeare()

Run the following to train and evaluate two models with basic features:

In [9]:
X,y,vocab = createBasicFeatures(corpus)
runEvaluation(X, y, vocab)

----------L1 Norm-----------
The model's average accuracy is 0.615385
The most informative terms for pos are: ['you', 'duke', 'prospero', 'i', 'helena', 'sir', 'leontes', 'a', 'private', 'preserving', 'preservers', 'preserver', 'preserved', 'preserve', 'preservative', 'president', 'preservation', 'presents', 'presentment', 'presently']
The most informative terms for neg are: ['him', 's', 'iago', 'imogen', 'o', 'brutus', 'lear', 'and', 'rom', 'ham', 'the', 'preserving', 'preservers', 'preserver', 'pretext', 'preserved', 'pretending', 'president', 'preservative', 'preservation']
----------L2 Norm-----------
The model's average accuracy is 0.769231
The most informative terms for pos are: ['i', 'you', 'duke', 'prospero', 'a', 'helena', 'your', 'antonio', 'sir', 'leontes', 'hermia', 'for', 'lysander', 'ariel', 'sebastian', 'demetrius', 'camillo', 'stephano', 'me', 'parolles']
The most informative terms for neg are: ['iago', 'othello', 's', 'him', 'imogen', 'what', 'lear', 'brutus', 'his', '

Run the following to train and evaluate two models with features that are interesting for distinguishing comedy and tragedy:

In [112]:
X,y,vocab = createInterestingFeatures(corpus)
runEvaluation(X, y, vocab)

----------L1 Norm-----------
The model's average accuracy is 0.538462
The most informative terms for pos are: ['never', 'sebastian', 'love', 'leontes', 'duke', 'say', 'antipholus', 'well', 'sweet', 'parolles', 'antonio', 'sir', 'relate', 'rejoicing yet ere', 'rejoicingly merry', 'rejoicing yet', 'rejoicing see profess', 'rejoicingly merry clothes', 'rejoicing see', 'rejoicingly']
The most informative terms for neg are: ['noble', 'nurse', 'queen', 'iago', 'achilles', 'lear', 'caesar', 'macbeth', 'timon', 'thy', 'othello', 'release brother', 'rejoicingly merry', 'rejoicingly', 'rejoicing yet ere', 'releas isabel', 'rejoicing see profess', 'rejoicing see', 'rejoicing ignorant promised', 'rejoicing ignorant']
----------L2 Norm-----------
The model's average accuracy is 0.730769
The most informative terms for pos are: ['sebastian', 'duke', 'sir', 'parolles', 'antipholus', 'love', 'would', 'antonio', 'leontes', 'one', 'say', 'never', 'well', 'sweet', 'ariel', 'therefore', 'signior', 'helena'

## **Observations**

When used the basic features, the importance was given to proper nouns, pronouns, verbs like is, which do not identify or distinguish between two genres. 
But still, the accuracy observed with basic features was 61.5% for L1 Norm and 76.9% for L2 norm.

For interesting features, after performing lemmatization and removing proper nouns, nouns, prepositions, foreign words and using uni,bi and tri-grams,
the important features are slightly relatable to the genre but surprisingly, the accuracy decreased.